In [7]:
"""
Plot voltages from a model_run.py output file
"""
%matplotlib qt
import os
import matplotlib
matplotlib.use('Qt4Agg')
import pickle
import numpy as np
from cnmodel.util import sound
import matplotlib.pyplot as mpl
from pathlib import Path
import pylibrary.PlotHelpers as PH
from cnmodel.util import vector_strength
import json

nreps = 5

# fnb = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS.p'
# fnbo = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS_noPosixPath.p'
# VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p

fnb = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"

fnb2 = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"
print(os.getcwd())

if not Path(fnb).is_file():
    print('not a file? ', fnb)
    assert False

if not Path(fnb2).is_file():
    print('not a file? ', fnb2)
    assert False

def get_one(fn):
    d = pickle.load(open(fn, 'rb'))
#     print(d.keys())
#     print('NTrials: ', len(d['trials']))
#     print(d['Params'].keys())
    return d

d = get_one(fnb)
# print(d['trials'][0]['inputSpikeTimes'])
# print(d)

for v in ['initIVStateFile', 'initANStateFile', 'simulationFilename', 'hocfile']:
    d['Params'][v] = str(d['Params'][v])
# print(d['Params'])
# with(open(fnbo, 'wb')) as fh:
#     pickle.dump(d, fh)



/Users/pbmanis/Desktop/Python/VCNModel


/Users/pbmanis/anaconda/envs/py3mpl3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import sys


In [4]:
def make_sound(stimInfo):
    if isinstance(stimInfo['pip_start'], list):
        pips = stimInfo['pip_start']
    else:
        pips = [stimInfo['pip_start']]
    if stimInfo['soundtype'] == 'tonepip':
        stim = sound.TonePip(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    elif stimInfo['soundtype'] == 'SAM':
        stim = sound.SAMTone(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], 
                          fmod=stimInfo['fmod'], dmod=stimInfo['dmod'],
                          pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    else:
        raise ValueError('StimInfo sound type %s not implemented' % stimInfo['soundtype'])

    stimWaveform = stim.generate()
    stimTimebase = stim.time
    return(stimTimebase, stimWaveform)

In [5]:
 def computeVS(data, andata=False, anf=0):
    allst = []
    if not andata:
        for trial in data['trials']:
            # print (trial)
            allst.extend(np.array(data['trials'][trial]['spikeTimes'])/1000.)
    else:
        for trial in data['trials']:
            for n in range(len(data['trials'][trial]['inputSpikeTimes'])):
                    allst.extend(np.array(data['trials'][trial]['inputSpikeTimes'][n])/1000.)
    allst = np.array(allst)
    allst = np.sort(allst) 
    si = d['trials'][0]['stimInfo']
    if si['soundtype'] == 'SAM':  # calculate vs and plot histogram
        # print('allst: ', allst)
        phasewin = [data['Params']['pip_start'][0] + 0.25*data['Params']['pip_duration'], 
            data['Params']['pip_start'][0] + data['Params']['pip_duration']]
        # print (phasewin)
        spkin = allst[np.where(allst > phasewin[0])]
        spikesinwin = spkin[np.where(spkin <= phasewin[1])]

        # set freq for VS calculation
        if si['soundtype'] == 'tone':
            f0 = data['Params']['F0']
            print ("Tone: f0=%.3f at %3.1f dbSPL, cell CF=%.3f" % 
                    (data['Params']['f0'], data['Params']['dB'], data['Params']['F0'] ))
        if si['soundtype'] == 'SAM':
            f0 = data['Params']['fmod']
            tstring = ("SAM Tone: f0=%.3f at %3.1f dbSPL, fMod=%3.1f  dMod=%5.2f, cell CF=%.3f" %
                 (data['Params']['F0'], data['Params']['dB'], data['Params']['fmod'], data['Params']['dmod'], data['Params']['F0']))
            print(tstring)
            # P.figure_handle.suptitle(tstring + '\n' + self.fn[self.Params['patterns'][0]].replace('_', '\_'), fontsize=9)
       # print('spikes: ', spikesinwin)
        print((' Sound type: ', si['soundtype']))
        if len(spikesinwin) < 10:
            vs = 0.
            print('AN Vector Strength: Insufficient spike count, n = %d' % (len(spikesinwin)))
        else:
            vs = vector_strength(spikesinwin*1000., f0)  # vs expects spikes in msec
            print('AN Vector Strength at %.1f: %7.3f, d=%.2f (us) Rayleigh: %7.3f  p = %.3e  n = %d' % (f0, vs['r'], vs['d']*1e6, vs['R'], vs['p'], vs['n']))
#             vspfile.add_data(self.Params['patterns'][0], self.Params['spirou'], vs['r'])
        return(vs)
    else:
        return 0

In [6]:
# f, ax = mpl.subplots(3,1, figsize=(6,6))
P = PH.regular_grid(4, 1, figsize=(6,6),
                verticalspacing=0.05, horizontalspacing=0.08,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
P.figure_handle.suptitle('AN PSTH 16 kHz, 30dbSPL, MS fibers, 50 reps, VCNc09')
ax = P.axarr.ravel()
inps = ['self', 'VCN c10']
for iax, fn in enumerate([fnb, fnb2]):
    d = get_one(fn)
#     print(d['trials'][0].keys())
    st = []
    print(len(d['trials']))
    for i in list(d['trials'].keys()):
#         for ansp in d['trials'][i]['spikeTimes']:
#             print(ansp)
        st.extend(d['trials'][i]['spikeTimes'])
#         print(d['trials'][i])
    st = np.array(st).ravel()
    ax[iax].hist(st, bins=np.arange(0, 250., 0.5))
    ax[iax].set_ylim((0,35))
    PH.referenceline(ax[iax], 20)
    ax[iax].set_ylabel('spike count')

    PH.nice_plot(ax[iax])
    vs = computeVS(d)
    vsin = computeVS(d, andata=True)
    txtlab = f"Input: {inps[iax]:s}  VS={vs['r']:.3f}, N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
    txtlab += f"  AN vs: {vsin['r']:.3f}"
    ax[iax].text(x=0.05, y=0.85,
                 s=txtlab,
                 transform=ax[iax].transAxes)
t, w = make_sound(d['trials'][0]['stimInfo'])
ax[3].plot(t*1000, w, linewidth=0.5)
ax[3].set_xlabel('T (ms)')

# get AN data from one fiber:
ansp = []
for trial in d['trials']:
            # print (trial)
    ansp.extend(np.array(d['trials'][trial]['inputSpikeTimes'][0])/1000.)
    print(len(d['trials'][trial]['inputSpikeTimes'][0]))
ax[2].hist(np.array(ansp)*1000., bins=np.arange(0, 250., 0.5))
ax[2].set_ylabel('AN spike count')           

PH.nice_plot(ax[iax])
for i in range(0,4):
    ax[i].set_xlim(75, 225)
#mpl.show()
P.figure_handle.tight_layout()
mpl.show()                    

50
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 200.0:   0.845, d=443.29 (us) Rayleigh: 591.389  p = 1.031e-217  n = 700
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 200.0:   0.659, d=657.23 (us) Rayleigh: 2637.758  p = 0.000e+00  n = 4003
50
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 200.0:   0.888, d=376.42 (us) Rayleigh: 604.813  p = 5.237e-234  n = 681
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength at 200.0:   0.657, d=658.94 (us) Rayleigh: 3304.254  p = 0.000e+00  n = 5028
12
13
18
13
7
14
14
15
13
21
15
15
18
16
18
16
14
14
18
9
15
16
12
14
13
18
13
15
19
15
12
16
12
18
18
19
16
17
16
16
14
12
11
17
15
14
15
17
14
17


/Users/pbmanis/anaconda/envs/py3mpl3/lib/python3.6/site-packages/matplotlib/figure.py:2362: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [ ]:
# PLOT FOR AN INPUT (one spike train)
# f, ax = mpl.subplots(3,1, figsize=(8,6))
P = PH.regular_grid(2, 1, figsize=(6,4),
                verticalspacing=0.2, horizontalspacing=0.08,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
f.suptitle('AN PSTH 16 kHz, 30dbSPL, MS fibers, 50 reps, AN SPikes')
ax = P.axarr.ravel()
inps = ['self', 'VCN c10']
for iax, fn in enumerate([fnb]): #, fnb2]):
    d = get_one(fn)
    print(d['trials'][0].keys())
    st = []
    for i in list(d['trials'].keys()):
        for ansp in d['trials'][i]['spikeTimes']:
            st.extend(ansp)
    st = np.array(st).ravel()
    ax[iax].hist(st, bins=np.arange(0, 250., 0.5))
    ax[iax].set_ylim((0,75))
    PH.referenceline(ax[iax], 25)
    ax[iax].set_ylabel('spike count')

    PH.nice_plot(ax[iax])
    vs = computeVS(d)
    ax[iax].text(x=0.05, y=0.85,
                 s=f"Input: {inps[iax]:s}  VS={vs['r']:.3f}, N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}",
                 transform=ax[iax].transAxes)
t, w = make_sound(d['trials'][0]['stimInfo'])
ax[2].plot(t*1000, w, linewidth=0.5)
ax[2].set_xlabel('T (ms)')

PH.nice_plot(ax[iax])
for i in range(0,3):
    ax[i].set_xlim(75, 225)
#mpl.show()
P.figure_handle.tight_layout()
mpl.show()